In [1]:
from __future__ import print_function, division
import os
import torch
import json
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import pdb

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from io import BytesIO
import urllib
from PIL import Image


In [5]:
path = '/home/edwin/Datasets/competitions/imaterialist-challenge-furniture-2018/'

In [3]:
num_epochs = 2

In [6]:
def get_image_id(x): return x['image_id']
def get_url(x): return x['url'][0]
def get_label_id(x): return x['label_id']

In [7]:
train_path = f'{path}train.json'

In [8]:
with open(train_path) as d1:
    train_json = json.load(d1)

In [ ]:
labels = list(map(get_label_id, train_json['annotations']))

In [ ]:
# of unique labels
classes = set(labels)
num_classes = len(classes)

In [12]:
images_sample = train_json['images']
annotations_sample = train_json['annotations']

In [13]:
ids = list(map(get_image_id, images_sample))
images = list(map(get_url, images_sample))
labels = list(map(get_label_id, annotations_sample))

In [14]:
data = {'image_id': ids, 'images': images, 'labels': labels}

In [15]:
data

{'image_id': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  15

In [16]:
df = pd.DataFrame(data=data)

In [19]:
df.to_pickle('df_full.pkl')

In [ ]:
df.to_pickle('imaterialist_full.pkl')

In [ ]:
df = pd.read_pickle('imaterialist_full.pkl')

In [ ]:
len(df)

In [ ]:
np.random.randn

In [ ]:
# creating a random dataset from pandas df
msk = np.random.rand(len(df)) < 0.05

In [ ]:
msk

In [ ]:
len(msk)

In [ ]:
train = df[msk]

In [ ]:
len(train)

In [ ]:
train

In [20]:
df1 = pd.read_pickle("df1.pkl")

In [21]:
IMAGES_TRAIN_PATH = "/home/edwin/Datasets/imaterialist-challenge-furniture-2018-sample/train"

In [54]:
class IMaterialDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_id = self.df.iloc[idx].image_id
        try:
            file = f"{IMAGES_TRAIN_PATH}/{image_id}.jpg"
            img = Image.open(file)
            img = np.asarray(img) 
        except Exception as error:
            img = np.zeros([800, 800, 3])
            
        label = self.df['labels'].iloc[idx]
        sample = {'image': img, 'label': label}
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample

In [55]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (74, 74), mode='constant')

        return {'image': img, 'label': label}
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        
        return {'image': torch.from_numpy(image).float(),
                'label': torch.from_numpy(np.array([label]))}
    

In [56]:
composed = transforms.Compose([Rescale(50), ToTensor()])

In [57]:
ds = IMaterialDataset(df=df1, transform=composed)

In [45]:
len(ds)

381

In [74]:
dataloader = DataLoader(ds, batch_size=4, shuffle=True, num_workers=1)

In [75]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [76]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 10, 5)
        self.fc1 = nn.Linear(2250 , 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 128)
        
    def forward(self, x):
        # original input torch.Size([4, 3, 75, 75])
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [77]:
cnn = CNNModel()

In [78]:
cnn.cuda()

CNNModel(
  (conv1): Conv2d (3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv2): Conv2d (6, 10, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=2250, out_features=1000)
  (fc2): Linear(in_features=1000, out_features=500)
  (fc3): Linear(in_features=500, out_features=128)
)

In [79]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)

In [82]:
iter = 0
for i, data in enumerate(dataloader, 0):
    images = Variable(data['image'].cuda())
    labels = Variable(data['label'].cuda())
    
    optimizer.zero_grad()
    
    outputs = cnn(images)
    loss = criterion(outputs, labels.view(-1))
    loss.backward()
    optimizer.step()
    iter += 1
    if iter % 5 == 0:
        correct = 0
        total = 0
        for j, data_train in enumerate(dataloader, 0):
            images = Variable(data_train['image'].cuda())
            labels = data_train['label']
            
            outputs = cnn(images)
            
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted.cpu() == labels.view(-1)).sum()
        accuracy = 100 * correct / total
        print('Correct:{}, Total:{}'.format(correct, total))
        print('Iteraton:{}, Loss:{}, Accuracy: {}'.format(iter, loss.data[0], accuracy))
        

Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f04465430b8>>
Traceback (most recent call last):
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 250, in __del__
    self._shutdown_workers()
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 246, in _shutdown_workers
    self.index_queue.put(None)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/queues.py", line 355, in put
    self._writer.send_bytes(obj)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
OSError: [Er

Correct:381, Total:381
Iteraton:5, Loss:4.750794887542725, Accuracy: 100.0


Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f04465430b8>>
Traceback (most recent call last):
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 250, in __del__
    self._shutdown_workers()
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 246, in _shutdown_workers
    self.index_queue.put(None)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/queues.py", line 355, in put
    self._writer.send_bytes(obj)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
OSError: [Er

Correct:381, Total:381
Iteraton:10, Loss:4.665383815765381, Accuracy: 100.0


Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f04465430b8>>
Traceback (most recent call last):
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 250, in __del__
    self._shutdown_workers()
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 246, in _shutdown_workers
    self.index_queue.put(None)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/queues.py", line 355, in put
    self._writer.send_bytes(obj)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
OSError: [Er

...

Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f04465430b8>>
Traceback (most recent call last):
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 250, in __del__
    self._shutdown_workers()
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 246, in _shutdown_workers
    self.index_queue.put(None)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/queues.py", line 355, in put
    self._writer.send_bytes(obj)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
OSError: [Er

Correct:381, Total:381
Iteraton:70, Loss:0.0, Accuracy: 100.0


Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7f04465430b8>>
Traceback (most recent call last):
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 250, in __del__
    self._shutdown_workers()
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 246, in _shutdown_workers
    self.index_queue.put(None)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/queues.py", line 355, in put
    self._writer.send_bytes(obj)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
OSError: [Er

KeyboardInterrupt: 

  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
Traceback (most recent call last):
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  F

In [ ]:
_, predicted = torch.max(outputs.data, 1)

In [ ]:
predicted

In [ ]:
sample['label']

In [ ]:
(predicted == sample['label']).sum()

In [ ]:
labels.view(-1)

In [ ]:
labels.view(-1)

In [ ]:
dataiter = iter(dataloader)

In [ ]:
sample = dataiter.next()

In [ ]:
images = Variable(sample['image'])

In [ ]:
images.shape

In [ ]:
outputs = cnn(images)

In [ ]:
labels = Variable(sample['label'].view(4))